In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
pd.set_option('display.float_format','{:.2f}'.format)

In [ ]:
tp_12_1 = '24_jun_12'
tp_12_2 = '25_jun_12'
tp_9_1 = '24_jun_9'
tp_9_2 = '25_jun_9'

In [ ]:
usag_3  = pd.read_parquet(f"Outputs/usag_diff_{tp_9_2}_{tp_12_2}_bucket_isocos_peinc_updated.parquet")

In [ ]:
# usag_3['macs_id'] = usag_3['global_id'].str.extract(r'^([A-Z]{2,3}\d{3})')

In [ ]:
usag_3['flag_gl_id'].value_counts()

In [ ]:
usag_3.columns

In [ ]:
usag_3

In [ ]:
# usag_3[usag_3['diff_Total_esales_curr']!=0].groupby('flag_gl_id')['macs_id'].nunique()

In [ ]:
usag_3

In [ ]:
core_segments = ['Growing','Stable','Declining']

In [ ]:
usag_df = usag_3[
    ((usag_3[f'{tp_12_1}_{tp_12_2}_act_mon_esales_sd_1_segment'].isin(core_segments)) &
     (usag_3[f'{tp_12_1}_{tp_12_2}_act_mon_nonesales_sd_1_segment'].isin(core_segments))) & 
     ~((usag_3[f'{tp_12_1}_{tp_12_2}_act_mon_esales_sd_1_segment']=='Stable') & (usag_3[f'{tp_12_1}_{tp_12_2}_act_mon_nonesales_sd_1_segment']=='Stable'))
]

In [ ]:
usag_df

In [ ]:
usag_df['diff_esales_incremental_abs_updated'].sum()

In [ ]:
# usag_df.to_excel('GSD Data.xlsx')

In [ ]:
def seg_num(df,tp_12_1,tp_12_2,core_segments):
    e_seg = f'{tp_12_1}_{tp_12_2}_act_mon_esales_sd_1_segment'
    n_seg = f'{tp_12_1}_{tp_12_2}_act_mon_nonesales_sd_1_segment'
    
    cond = [
        (df[e_seg] == seg1) & (df[n_seg] == seg2)
        for seg1 in core_segments for seg2 in core_segments
    ]

    choices = [num for num in range(1,10)]

    return np.select(cond,choices,default=0)

def channel_shift_12(df,tp_12_1,tp_12_2):
    tp1_e_total = f'{tp_12_1}_Total_esales'
    tp1_n_total = f'{tp_12_1}_Total_nonesales'
    seg_nums_not_included = [2,4,5,6,8,9]
    
    cond1 = (df[tp1_e_total] + df[tp1_n_total] < 1) | (df['seg_num'].isin(seg_nums_not_included))
    cond2 = ((df['diff_12_esales'] < 0) & (df['diff_12_nonesales'] < 0)) | ((df['diff_12_esales'] > 0) & (df['diff_12_nonesales'] > 0) & (df['diff_12_esales'] < df['diff_12_nonesales'])) 

    cond = [cond1, cond2]

    choices = ['Not Considered','No Channel Shift']

    return np.select(cond, choices, default='Channel Shift')

def channel_shift_9(df,tp_9_1,tp_9_2):
    tp1_e_total = f'{tp_9_1}_Total_esales'
    tp1_n_total = f'{tp_9_1}_Total_nonesales'
    seg_nums_not_included = [2,4,5,6,8,9]
    
    cond1 = (df[tp1_e_total] + df[tp1_n_total] < 1) | (df['seg_num'].isin(seg_nums_not_included))
    cond2 = ((df['diff_9_esales'] < 0) & (df['diff_9_nonesales'] < 0)) | ((df['diff_9_esales'] > 0) & (df['diff_9_nonesales'] > 0) & (df['diff_9_esales'] < df['diff_9_nonesales'])) 

    cond = [cond1, cond2]

    choices = ['Not Considered','No Channel Shift']

    return np.select(cond, choices, default='Channel Shift')

def compute_discount_12(df,tp_12_1):
    bucket_col = f'{tp_12_1}_bucket'
    comp_yoy_col = 'comp_bucket_YoY_12'

    conditions = [
        df[bucket_col].isin(['<0', '0'])
    ]
    
    choices = [
        0.0
    ]
    
    comp_doll = np.select(conditions, choices, default = df[comp_yoy_col] * df[f'{tp_12_1}_Total_esales'])
    return comp_doll

def compute_discount_9(df,tp_9_1):
    bucket_col = f'{tp_9_1}_bucket'
    comp_yoy_col = 'comp_bucket_YoY_9'

    conditions = [
        df[bucket_col].isin(['<0', '0'])
    ]
    
    choices = [
        0.0
    ]
    
    comp_doll = np.select(conditions, choices, default = df[comp_yoy_col] * df[f'{tp_9_1}_Total_esales'])
    return comp_doll

In [ ]:
usag_df['diff_12_esales'] = usag_df[f'{tp_12_2}_Total_esales'] - usag_df[f'{tp_12_1}_Total_esales'] 
usag_df['diff_12_nonesales'] = usag_df[f'{tp_12_2}_Total_nonesales'] - usag_df[f'{tp_12_1}_Total_nonesales'] 

usag_df['diff_9_esales'] = usag_df[f'{tp_9_2}_Total_esales'] - usag_df[f'{tp_9_1}_Total_esales'] 
usag_df['diff_9_nonesales'] = usag_df[f'{tp_9_2}_Total_nonesales'] - usag_df[f'{tp_9_1}_Total_nonesales'] 

In [ ]:
usag_df['seg_num'] = seg_num(usag_df,tp_12_1, tp_12_2, core_segments)

In [ ]:
usag_df['Channel_Shift_Status_12'] = channel_shift_12(usag_df,tp_12_1, tp_12_2)

usag_df['Channel_Shift_Status_9'] = channel_shift_9(usag_df,tp_9_1, tp_9_2)

In [ ]:
usag_df['esales_l12_12%'] = (usag_df[f'{tp_12_2}_Total_esales'] /( usag_df[f'{tp_12_2}_Total_esales'] + usag_df[f'{tp_12_2}_Total_nonesales'])).fillna(0)
usag_df['esales_l1324_12%'] = (usag_df[f'{tp_12_1}_Total_esales'] /( usag_df[f'{tp_12_1}_Total_esales'] + usag_df[f'{tp_12_1}_Total_nonesales'])).fillna(0)
usag_df['esales_tobe_12'] = ((usag_df[f'{tp_12_1}_Total_esales'] /( usag_df[f'{tp_12_1}_Total_esales'] + usag_df[f'{tp_12_1}_Total_nonesales'])) * ( usag_df[f'{tp_12_2}_Total_esales'] + usag_df[f'{tp_12_2}_Total_nonesales'])).fillna(0)


usag_df['esales_l12_9%'] = (usag_df[f'{tp_9_2}_Total_esales'] /( usag_df[f'{tp_9_2}_Total_esales'] + usag_df[f'{tp_9_2}_Total_nonesales'])).fillna(0)
usag_df['esales_l1324_9%'] = (usag_df[f'{tp_9_1}_Total_esales'] /( usag_df[f'{tp_9_1}_Total_esales'] + usag_df[f'{tp_9_1}_Total_nonesales'])).fillna(0)
usag_df['esales_tobe_9'] = ((usag_df[f'{tp_9_1}_Total_esales'] /( usag_df[f'{tp_9_1}_Total_esales'] + usag_df[f'{tp_9_1}_Total_nonesales'])) * ( usag_df[f'{tp_9_2}_Total_esales'] + usag_df[f'{tp_9_2}_Total_nonesales'])).fillna(0)

In [ ]:
usag_df['Channel_shift_tobe_12'] = np.select([(usag_df[f'{tp_12_1}_Total_esales'] + usag_df[f'{tp_12_1}_Total_nonesales'] < 1)],[0],default=(usag_df[f'{tp_12_2}_Total_esales'] - usag_df['esales_tobe_12']))

usag_df['Channel_shift_tobe_9'] = np.select([(usag_df[f'{tp_9_1}_Total_esales'] + usag_df[f'{tp_9_1}_Total_nonesales'] < 1)],[0],default=(usag_df[f'{tp_9_2}_Total_esales'] - usag_df['esales_tobe_9']))

In [ ]:
usag_df['Channel_Shift_value_12'] = np.select([(usag_df['Channel_Shift_Status_12']=='Channel Shift')],[(usag_df['Channel_shift_tobe_12'])],default=0)

usag_df['Channel_Shift_value_9'] = np.select([(usag_df['Channel_Shift_Status_9']=='Channel Shift')],[(usag_df['Channel_shift_tobe_9'])],default=0)

In [ ]:
usag_df['Channel_Shift_value_12'].sum()

In [ ]:
usag_df['Channel_Shift_value_9'].sum()

In [ ]:
usag_df['Channel_Shift_Value_12-9'] = usag_df['Channel_Shift_value_12'] - usag_df['Channel_Shift_value_9']

In [ ]:
usag_df['Channel_Shift_Value_12-9'].sum()

In [ ]:
usag_df[['global_id','Channel_Shift_Value_12-9']].to_parquet('../USAG/channel shift.parquet')

In [ ]:
usag_df['Incremental_adjusted'] = usag_df['diff_esales_incremental_abs_updated'] - usag_df['Channel_Shift_Value_12-9']

In [ ]:
usag_df['Incremental_adjusted'].sum()

In [ ]:
usag_df.groupby('flag_gl_id')['Incremental_adjusted'].sum()

In [ ]:
usag_df.groupby('flag_gl_id')['global_id'].count()

In [ ]:
usag_df.groupby(f'{tp_12_1}_bucket')['Incremental_adjusted'].sum()

In [ ]:
usag_df.groupby(f'{tp_12_1}_bucket')['global_id'].count()

In [ ]:
# usag_df.groupby('flag_gl_id')['macs_id'].nunique()

In [ ]:
# cos_stores = list(usag_df.groupby('flag_gl_id')['macs_id'].unique()['COMPANY_OWNED'])
# rev = usag_3['diff_Total_esales_curr'].sum()
# tot_rev = usag_3[usag_3['macs_id'].isin(cos_stores)]['diff_Total_esales_curr'].sum()
# gsd_rev = usag_df[usag_df['macs_id'].isin(cos_stores)]['diff_Total_esales_curr'].sum()
# print(rev,tot_rev,gsd_rev)

In [ ]:
cross_tab_view = pd.crosstab(usag_df[f'{tp_12_1}_{tp_12_2}_act_mon_esales_sd_1_segment'],usag_df[f'{tp_12_1}_{tp_12_2}_act_mon_nonesales_sd_1_segment'],
            values=usag_df['Channel_Shift_Value_12-9'],aggfunc='sum')
cross_tab_view = cross_tab_view.reindex(index=core_segments,columns=core_segments)
cross_tab_view

In [ ]:
cross_tab_view2 = pd.crosstab(usag_df[f'{tp_12_1}_{tp_12_2}_act_mon_esales_sd_1_segment'],usag_df[f'{tp_12_1}_{tp_12_2}_act_mon_nonesales_sd_1_segment'],
            values=usag_df['diff_esales_incremental_abs_updated'],aggfunc='sum',margins=True)
cross_tab_view2 = cross_tab_view2.reindex(index=core_segments,columns=core_segments)
cross_tab_view2

In [ ]:
usag_3['diff_Total_esales_curr'].sum()

In [ ]:
usag_df.to_parquet(f'Outputs/{tp_12_2} ISO COS Channel Shift Discount.parquet')

In [ ]:
# usag_df.to_excel(f'Outputs/{tp_12_2} ISO COS Channel Shift Discount.xlsx')

In [ ]:
# pd.DataFrame(cos_stores).to_excel('cos_stores.xlsx')

In [ ]:
cross_tab_view.to_excel(f'Outputs/{tp_12_2} ISO COS Channel Shift crosstab view.xlsx')

In [ ]:
usag_df.groupby([f'{tp_12_1}_bucket','flag_gl_id'])['Incremental_adjusted'].sum().reset_index().to_excel('Inc_bucketwise.xlsx')

In [ ]:
usag_df.groupby([f'{tp_12_1}_bucket','flag_gl_id'])['Incremental_adjusted'].sum().reset_index()

In [ ]:
# bucket_bins = [-np.inf,0,0.001,500,1000,2500, 5000, 10000, 25000, np.inf]
# buckets = ['<0','0','0-500','500-1000','1000-2500','2500-5000', '5000-10000', '10000-25000', '25000+']

# usag_df[f'{tp_12_1}_bucket'] = pd.cut(usag_df[f'{tp_12_1}_Total_esales'], bins=bucket_bins, labels=buckets, right=False)
# usag_df[f'{tp_9_1}_bucket'] = pd.cut(usag_df[f'{tp_9_1}_Total_esales'], bins=bucket_bins, labels=buckets, right=False)

In [ ]:
# esales_buckets_12 = usag_df.groupby(f'{tp_12_1}_bucket')[[f'{tp_12_1}_Total_esales',f'{tp_12_2}_Total_esales']].sum().reset_index()
# esales_buckets_12['comp_bucket_YoY_12'] = (esales_buckets_12[f'{tp_12_2}_Total_esales']/esales_buckets_12[f'{tp_12_1}_Total_esales'])-1
# esales_buckets_12 = esales_buckets_12[[f'{tp_12_1}_bucket','comp_bucket_YoY_12']]
# usag_df = usag_df.merge(esales_buckets_12,how='left',on=f'{tp_12_1}_bucket')
# esales_buckets_12

In [ ]:
# esales_buckets_9 = usag_df.groupby(f'{tp_9_1}_bucket')[[f'{tp_9_1}_Total_esales',f'{tp_9_2}_Total_esales']].sum().reset_index()
# esales_buckets_9['comp_bucket_YoY_9'] = (esales_buckets_9[f'{tp_9_2}_Total_esales']/esales_buckets_9[f'{tp_9_1}_Total_esales'])-1
# esales_buckets_9 = esales_buckets_9[[f'{tp_9_1}_bucket','comp_bucket_YoY_9']]
# usag_df = usag_df.merge(esales_buckets_9,how='left',on=f'{tp_9_1}_bucket')
# esales_buckets_9

In [ ]:
# usag_df['comp_bucket_doll_12'] = compute_discount_12(usag_df,tp_12_1)

# usag_df['comp_bucket_doll_9'] = compute_discount_9(usag_df,tp_9_1)

In [ ]:
# usag_df['adj_tobe_12'] = usag_df['Channel_shift_tobe_12'] - usag_df['comp_bucket_doll_12']

# usag_df['adj_tobe_9'] = usag_df['Channel_shift_tobe_9'] - usag_df['comp_bucket_doll_9']


In [ ]:
# usag_df['adj_doll_12'] = np.select([(usag_df['Channel_Shift_Status_12']=='Channel Shift')],[(usag_df['adj_tobe_12'])],default=0)

# usag_df['adj_doll_9'] = np.select([(usag_df['Channel_Shift_Status_9']=='Channel Shift')],[(usag_df['adj_tobe_9'])],default=0)

In [ ]:
# usag_df['adj_doll_12'].sum() - usag_df['adj_doll_9'].sum()

In [ ]:
# usag_df['adj_doll_9'].sum()

In [ ]:
# usag_df.to_excel(f'Outputs/{tp_12_2}Channel Shift Discount.xlsx')